In [3]:
import pandas as pd
import numpy  as np
from pprint import pprint


## Goal

*To predict whether or not a student will get certified/explore their course based on features about that individual.*

In this project, I aim to classify whether or not an individual will explore an MOOC or will be certified. I am only going to include individuals that actually viewed the course content- not just registered for the course. This is because several individuals simply registered for tens of courses without ever viewing the content, and this will distort the efficacy of our model. 

The target variable, explored,  refers to the notion that the individual accessed at least half of the chapters of the offered course.  It also includes people that finished the course and were certified. 

I am choosing this as my target variable for two reasons:

1. Students often do not plan on becoming certified with MOOCs, so exploring extensively is a good proxy for engaged learning.   

2. The class imbalance of certified (95-5 split) is a bit infeasible. 

### Column Description

`course_id`: the course identifier.   
`userid_DI`: the user identifier.      
`registered`: whether the user registered for the course.   
`viewed`: anyone who accessed the 'Courseware tab' in the edX platform.   
`explored`: anyone who accessed at least half of the chapters in the courseware.    
`certified`: anyone who earned a certificate.    
`final_cc_cname_DI`: country name or continent/region name.    
`LoE`: highest level of education.    
`YoB`: year of birth.    
`gender`: male, female, or other.    
`grade`: final grade in the course.    
`start_time_DI:` date of course registration   
`last_event_DI:` date of last interaction in course. Blank if none beyond registration.    
`nevents`: number of interactions with the course (blank if done beyond registration).    
`ndays_act`: number of unique days student interacted with the course.    
`nplay_video`: number of play video events.   
`nchapters`: number of chapters within the Courseware with which the students interacted.   
`nforum_posts`: number of posts to the discussion forum.    
`roles`:  identifies staff and instructors.   
`incomplete flag`: identifies records that are internally inconsistent.  This includes missing tracking logs. 

# Data Cleaning

In [36]:
data = pd.read_csv('data/HMXPC13_DI_v2_5-14-14.csv')

In [5]:
data.head(2)

,course_id,userid_DI,registered,viewed,explored,certified,final_cc_cname_DI,LoE_DI,YoB,gender,grade,start_time_DI,last_event_DI,nevents,ndays_act,nplay_video,nchapters,nforum_posts,roles,incomplete_flag
0,HarvardX/CB22x/2013_Spring,MHxPC130442623,1,0,0,0,United States,NaN,NaN,NaN,0,2012-12-19,2013-11-17,NaN,9.0,NaN,NaN,0,NaN,1.0
1,HarvardX/CS50x/2012,MHxPC130442623,1,1,0,0,United States,NaN,NaN,NaN,0,2012-10-15,NaN,NaN,9.0,NaN,1.0,0,NaN,1.0


In [6]:
def check_proportion_nas(df):
    return sorted([(column, round(df[column].isna().mean(), 3)) for column in df.columns],\
                  key = lambda x:x[1])
    
        

In [7]:
check_proportion_nas(data)

[('course_id', 0.0),
 ('userid_DI', 0.0),
 ('registered', 0.0),
 ('viewed', 0.0),
 ('explored', 0.0),
 ('certified', 0.0),
 ('final_cc_cname_DI', 0.0),
 ('start_time_DI', 0.0),
 ('nforum_posts', 0.0),
 ('grade', 0.075),
 ('gender', 0.135),
 ('YoB', 0.151),
 ('LoE_DI', 0.165),
 ('ndays_act', 0.254),
 ('last_event_DI', 0.279),
 ('nevents', 0.311),
 ('nchapters', 0.404),
 ('nplay_video', 0.714),
 ('incomplete_flag', 0.844),
 ('roles', 1.0)]

For several of the columns, the proportion of NA's is rather signficant.  The `role` is completely missing, so we will get rid of this.  The `incomplete_flag` has the next highest proportion, but this column simply denoted a clerical error when collecting the data, so we can safely get rid of it. 

Furthermore, I think it would be wise to drop a few more of these features since they are rather self-annotating. Basically, obviously if you have a high returned grade, then you would be certified.  Knowing this information will make the classification task much too simple. Let's focus more on demographic information. 

In [8]:
data.drop(columns = ['roles', 'nforum_posts', 'grade', 'ndays_act', \
                            'last_event_DI', 'nevents', 'nplay_video', 'incomplete_flag', \
                            'nchapters', 'registered'], axis = 1, inplace = True)

In [9]:
data = data.dropna()
data = data[data.viewed == 1]


In [10]:
data[data.userid_DI == 'MHxPC130027283']

,course_id,userid_DI,viewed,explored,certified,final_cc_cname_DI,LoE_DI,YoB,gender,start_time_DI
154924,HarvardX/CS50x/2012,MHxPC130027283,1,0,0,United States,Secondary,1992.0,f,2013-07-12
154925,HarvardX/ER22x/2013_Spring,MHxPC130027283,1,0,0,United States,Secondary,1992.0,f,2013-07-31
154926,HarvardX/PH207x/2012_Fall,MHxPC130027283,1,0,0,United States,Secondary,1992.0,f,2013-07-12
154927,HarvardX/PH278x/2013_Spring,MHxPC130027283,1,0,0,United States,Secondary,1992.0,f,2013-07-31
486335,MITx/14.73x/2013_Spring,MHxPC130027283,1,0,0,United States,Secondary,1992.0,f,2013-07-12
486336,MITx/2.01x/2013_Spring,MHxPC130027283,1,0,0,United States,Secondary,1992.0,f,2013-07-31
486337,MITx/3.091x/2012_Fall,MHxPC130027283,1,0,0,United States,Secondary,1992.0,f,2013-07-12
486338,MITx/3.091x/2013_Spring,MHxPC130027283,1,0,0,United States,Secondary,1992.0,f,2013-07-12
486339,MITx/6.002x/2012_Fall,MHxPC130027283,1,0,0,United States,Secondary,1992.0,f,2013-07-12
486340,MITx/6.002x/2013_Spring,MHxPC130027283,1,0,0,United States,Secondary,1992.0,f,2013-07-12


Certain users viewed massive amounts of courses without exploring anything as shown above.  This will have a negative impact on our modeling capabilities since we are attempting to predict whether or not a user explored a course based on class information and demographic information.  Because of this, I am going to only include unique user ids, and also ensure that if they have explored a course, to ensure that entry is reflected in the data. 

In [11]:
#ensure we keep all explored, then only keep the first unique user
data = data.sort_values(by=['explored', 'userid_DI'], ascending = False)\
                .drop_duplicates(subset = 'userid_DI', keep = 'first')


In [12]:
data.shape

(274044, 10)

In [13]:
data.head(2)

,course_id,userid_DI,viewed,explored,certified,final_cc_cname_DI,LoE_DI,YoB,gender,start_time_DI
561489,MITx/14.73x/2013_Spring,MHxPC130597663,1,1,1,United States,Bachelor's,1958.0,f,2013-02-12
515309,MITx/14.73x/2013_Spring,MHxPC130597647,1,1,1,United States,Bachelor's,1987.0,f,2012-12-28


In [14]:
data['explored/cert'] = data.explored | data.certified


There are only 4 out of our nearly 300,000 samples that consider themselves as "other" for their gender so I will drop them for this model.

In [15]:
data = data[data.gender!='o']

In [16]:
data.gender.value_counts()

m    206740
f     67300
Name: gender, dtype: int64

In [17]:
data.set_index(np.arange(len(data)), inplace = True)

In [18]:
data.head(2)

,course_id,userid_DI,viewed,explored,certified,final_cc_cname_DI,LoE_DI,YoB,gender,start_time_DI,explored/cert
0,MITx/14.73x/2013_Spring,MHxPC130597663,1,1,1,United States,Bachelor's,1958.0,f,2013-02-12,1
1,MITx/14.73x/2013_Spring,MHxPC130597647,1,1,1,United States,Bachelor's,1987.0,f,2012-12-28,1


In [19]:
#let's clean up our courses so they are cleaner when we plot them

#Harvard courses
data['CS50X'] = data.course_id.apply(lambda x:  'CS50X' if 'CS' in x else '')
data['HealthStat'] = data.course_id.apply(lambda x:  'HealthStat' if 'PH207' in x else '')
data['HeroesX'] = data.course_id.apply(lambda x: 'HeroesX' if 'CB22' in x else '')
data['JusticeX'] = data.course_id.apply(lambda x: 'JusticeX' if 'ER22' in x else '')
data['HealthEnv'] = data.course_id.apply(lambda x:  'HealthEnv' if 'PH278' in x else '')


#MIT courses
data['CS-1'] = data.course_id.apply(lambda x: 'CS-1' if '6.00x/2012'  in x else '')
data['CS-2'] = data.course_id.apply(lambda x: 'CS-2' if '6.00x/2013'  in x else '')
data['Circuits-1'] = data.course_id.apply(lambda x: 'Circuits-1' if '6.002x/2012'  in x else '')
data['Circuits-2'] = data.course_id.apply(lambda x: 'Circuits-2' if '6.002x/2013'  in x else '')
data['SSChem-1'] = data.course_id.apply(lambda x: 'SSChem-1' if '3.091x/2012'  in x else '')
data['SSChem-2'] = data.course_id.apply(lambda x: 'SSChem-2' if '3.091x/2013'  in x else '')
data['Poverty'] = data.course_id.apply(lambda x: 'Poverty' if '14.73x'  in x else '')
data['E&M'] = data.course_id.apply(lambda x: 'E&M' if '8.02x'  in x else '')
data['Biology'] = data.course_id.apply(lambda x: 'Biology' if '7.00x'  in x else '')
data['Structures'] = data.course_id.apply(lambda x: 'Structures' if '2.01x'  in x else '')
data['MechRev'] = data.course_id.apply(lambda x: 'MechRev' if '8.MReV'  in x else '')



In [20]:
data['course'] = data['CS50X'] + data['HealthStat'] + data['HeroesX']\
               + data['JusticeX'] + data['HealthEnv'] + data['CS-1'] \
               + data['CS-2'] + data['Circuits-1'] + data['Circuits-2'] \
               + data['SSChem-1'] + data['SSChem-2'] + data['Poverty'] \
               + data['E&M'] + data['Biology'] + data['Structures']\
               + data['MechRev']

In [21]:
data.drop(columns= ['CS50X','HealthStat','HeroesX','JusticeX',\
                    'HealthEnv','CS-1','CS-2','Circuits-1',\
                    'Circuits-2','SSChem-1','SSChem-2' ,'Poverty'\
                    ,'E&M','Biology', 'Structures','MechRev'], inplace = True)

Let's group each person's age into bins to extract more information from the groupings. 

In [22]:
#don't include 70s and older in order to avoid multicollinearity
data['>=70'] = np.where(data.YoB <=1943, 1, 0)
data['60s'] = np.where((data.YoB > 1943) & (data.YoB <= 1953), 1, 0)
data['50s'] = np.where((data.YoB > 1953) & (data.YoB <= 1963), 1, 0)
data['40s'] = np.where((data.YoB > 1963) & (data.YoB <= 1973), 1, 0)
data['30s'] = np.where((data.YoB > 1973) & (data.YoB <= 1983), 1, 0)
data['25_to_30'] = np.where((data.YoB > 1983) & (data.YoB <= 1988), 1, 0)
data['20_to_25'] = np.where((data.YoB > 1988) & (data.YoB <= 1993), 1, 0)
data['under_20'] = (np.where(data.YoB > 1993, 1, 0))
data.drop(columns=['YoB'], inplace=True)

In [23]:
data['month_started'] = data.start_time_DI.apply(lambda x: x.split('-')[1])
data.drop(columns = ['start_time_DI'], inplace=True)

In [24]:
# let's create a column to denote whether or not the class was taken from MIT or Harvard
data['MIT'] = np.where(data.course_id.apply(lambda x: 'MIT' in x), 1, 0)

Looking at the course id documentation, I notice that certain courses are repeats of each other, whereas other courses there is only one offering.  I would like to make a column to denote this distinction.  Furthermore, it may be worthwhile to group each course into a cluster of subjects, such as computer science, life science, physics, etc.  

In [25]:
#note, I did not include one public health course to account for multicollinearity
course_number = data.course_id.apply(lambda x : x.split('/')[1])
data['cs'] = course_number.apply(lambda x: int(('CS' in x) or ('6.00x' in x)))
data['social'] = course_number.apply(lambda x: int(('14.73x' in x) or ('ER22x' in x)))
data['mechanics'] = course_number.apply(lambda x: int(('2.01x' in x) or ('MReV' in x)))
data['electrical'] = course_number.apply(lambda x: int(('6.002x' in x) or ('8.02x' in x)))
data['bio'] = course_number.apply(lambda x: int(('7.00x' in x)))
data['public_health'] = course_number.apply(lambda x: int(('PH207' in x)))

In [26]:
certified = data[data['explored/cert'] == 1]

In [28]:
#store to toy around with in Plotly in another notebook
data.to_csv('visualize.csv')

## Prepare Model for Flask

In [32]:
data.head()

,course_id,userid_DI,explored,certified,explored/cert,course,>=70,60s,50s,40s,...,month_started_03,month_started_04,month_started_05,month_started_06,month_started_07,month_started_08,month_started_09,month_started_10,month_started_12,gender_f
0,MITx/14.73x/2013_Spring,MHxPC130597663,1,1,1,Poverty,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,MITx/14.73x/2013_Spring,MHxPC130597647,1,1,1,Poverty,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,HarvardX/ER22x/2013_Spring,MHxPC130597622,1,0,1,JusticeX,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,HarvardX/CS50x/2012,MHxPC130597614,1,0,1,CS50X,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,MITx/6.002x/2013_Spring,MHxPC130597593,1,0,1,Circuits-2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


For my Flask application, I want to simplify my model so that it doesn't include every single feature.  This will make a more seamless integration without too many options for the user.  Let's prepare this data to export to a different csv. 

# Model Preparation

In [29]:
#viewed is 1 in every single row, carries no additional information
data.drop(columns = ['viewed'], axis = 1, inplace = True)

In [30]:
data =  pd.get_dummies(data,\
        columns = [ 'final_cc_cname_DI', 'LoE_DI',\
                   'month_started', 'gender'])\
                  .drop(columns = ['gender_m'])

In [31]:
data.drop(columns = ['final_cc_cname_DI_Unknown/Other', 'LoE_DI_Doctorate', \
                     'month_started_11'], inplace = True)

Finally, we also do not need the userid when it comes to modeling nor do we need the specfic course title since I want this analysis to generalize to more than specific course titles.  We can then save our dataframe into a csv and start modeling using various classification algorithms. 

In [33]:
data.drop(columns = ['userid_DI', 'course_id'], inplace = True)

In [34]:
data.to_csv('data/modeling_data.csv')